In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:

from IPython.display import HTML
from base64 import b64encode
video_path = '/content/drive/MyDrive/Konkuk_CV/영상3.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

In [ ]:
class Car_Detection():

    def __init__(self,path):
        self.path = path
        self.frames = list()
        self.detected_frames = list()
        self.cap = None

    def video_to_frames(self):
        self.cap = cv2.VideoCapture(self.path)
        while self.cap.isOpened():
            run, frame = self.cap.read()
            if not run:
                break
            img = cv2.cvtColor(frame, cv2.IMREAD_COLOR)
            resize_frame = cv2.resize(frame, (960, 480), interpolation=cv2.INTER_LINEAR)
            self.frames.append(resize_frame)

    def frames_rgb_to_hsv(self,frames):
        g_frames = []
        for frame in frames:
            g_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2HSV))

        return np.array(g_frames, dtype='uint8')

    def frames_hsv_to_rgb(self,frames):
        g_frames = []
        for frame in frames:
            g_frames.append(cv2.cvtColor(frame, cv2.COLOR_HSV2BGR))

        return np.array(g_frames, dtype='uint8')


    def connect_components(self,frames, original_frames, area_min):
        for k, frame in enumerate(frames):
            cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(frame)
            for i in range(1, cnt):
                (x, y, w, h, area) = stats[i]
                cent_x,cent_y = centroids[i]
                if area < area_min:
                    continue
                cv2.rectangle(original_frames[k], (x, y), (x+w, y+h), (0,0,255), 2)
        return original_frames

    def blur(self,frames):
        blur_frames = list()
        for k, frame in enumerate(frames):
            blur_frames.append(cv2.GaussianBlur(frame[:,:,2], (5, 5), 0))
        return np.array(blur_frames,dtype='uint8')

    def background_delete(self,frames,method):
        fg_masks = list()
        for frame in frames:
            fg_mask = method.apply(frame)
            fg_masks.append(fg_mask)
        return np.array(fg_masks,dtype='uint8')

    def detect_foreground(self,frames):
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE  , (3,3))
        binary_frames = list()
        for frame in frames:
            _, img_hand = cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)

            img_hand = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel,iterations=2)
            img_hand = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel,iterations=2)
            _, img_hand = cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)

            binary_frames.append(img_hand)

        return np.array(binary_frames,dtype='uint8')

    def fit(self):
        self.video_to_frames()
        fgbg = cv2.createBackgroundSubtractorMOG2(varThreshold = 10,detectShadows=True)
        bin = self.background_delete(self.frames,fgbg)
        bin_frames = self.detect_foreground(bin)
        self.detected_frames = self.connect_components(bin_frames,self.frames.copy(),600)

    def get_frames(self):
        return self.detected_frames

    def cvt_to_video(self,output_path):
        width = 960
        height = 480
        fps = self.cap.get(5)
        fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (int(width),int(height)))
        for frame in self.detected_frames:
            video_writer.write(frame)

        video_writer.release()
        self.cap.release()


In [ ]:
detector = Car_Detection('/content/drive/MyDrive/Konkuk_CV/영상3.mp4')
detector.fit()
detector.cvt_to_video('/content/drive/MyDrive/Konkuk_CV/output_car.mp4')